 # Dual Moving Average Crossover Trading Signal

 ---

 ### Import Libraries and Dependencies

In [4]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path

 ### Read source data

In [2]:
#API get request here

# Below assumes a dataframe of price information includes "date", and "close" columns.
# call create_signal_df(<your_dataframe>)
# returns a section of the resulting signals_df

 ### Generate a Dual Moving Average Crossover Trading Signal

In [2]:
def create_signal_df(input_df):
    # Grab just the `date` and `close` from the IEX dataset
    signals_df = input_df.loc[:, ["date", "close"]].copy()

    # Set the short window and long windows
    short_window = 50
    long_window = 200

    # Set the `date` column as the index
    signals_df = signals_df.set_index("date", drop=True)

    # Generate the short and long moving averages (50 and 200 days, respectively)
    signals_df["SMA50"] = signals_df["close"].rolling(window=short_window).mean()
    signals_df["SMA200"] = signals_df["close"].rolling(window=long_window).mean()
    signals_df["Signal"] = 0.0

    # Generate the trading signal 0 or 1,
    # where 0 is when the SMA50 is under the SMA200 (Bear Market), and
    # where 1 is when the SMA50 is higher (or crosses over) the SMA200 (Bull Market)
    signals_df["Signal"][short_window:] = np.where(
        signals_df["SMA50"][short_window:] > signals_df["SMA200"][short_window:], 1.0, 0.0
    )

    # if a column "bull/bear" is required the following can be used and adjusted to suit
    # Calculate the points in time at which a position should be taken, 1 or -1
    # signals_df["Bull/Bear"] = signals_df["Signal"].diff()

    return signals_df[200:250]

 ### Plot sentiment change using Dual Moving Average Crossover

In [3]:
# Visualize Bearish sentiment relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['close'].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize Bullish sentiment change relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['close'].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA50', 'SMA100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

NameError: name 'signals_df' is not defined